## Prorail project: Baseline Model

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta

from sklearn.linear_model import LinearRegression
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 

### Business understanding

##### Achtergrond:

**De doelstelling van het project is om de planners te helpen om de hervatting van het treinverkeer te organiseren. In de praktijk blijkt dat de voorspellingen van de monteurs vaak te pessimistisch zijn. De vraag van ProRail en de planners van de NS is of op basis van de data een betere voorspelling te maken is waardoor de dienstregeling sneller hervat kan worden. De voorspellingen moeten getoond worden in een dashboard waar relevante informatie over de voorspelling, zoals de betrouwbaarheid, getoond wordt.**

Er moet dus gekeken worden naar de tijd voor het oplossen van een probleem. Dit kan gedaan worden door als target het gemiddelde te pakken en daarmee een voorspelling te doen voor aankomende meldingen. We kunnen als vraag hebben:"De reparatietijd is altijd gelijk aan de gemiddelde reparatietijd."

Deze voorspelling moet helpen voor Reizigers, om ze een inschatting te geven hoe lang een probleem gaat duren. Om dat goed te visualiseren wordt eerst de voorspelling gegeven aan een reizigerplanner van de NS/ProRail, want tegenwoordig wordt die voorspelling gedaan met de losse vinger door de aannemers. Deze is meestal erg hoog ingeschat of laag ingeschat, wegens het opmerken/negeren van onverwachtse problemen die kunnen gebeuren. Dat geeft de reiziger veel verwarring en kost veel tijd en geld. Dat moet automatisch en beter kunnen.

##### Doelen:

Om de reparatietijd in te schatten kan je kijken naar de eerder voorkomende meldingen. In dit geval de 0.9 miljoen rijen aan meldingen data. Om daar een beginnetje te maken, gaan we de vraag beantwoorden: "De reparatietijd is altijd gelijk aan de gemiddelde reparatietijd.". Als je de betrouwbaarheid berekend kan je dat gebruiken om accurater de reparatietijd te bepalen.
Toch zal dit op elke plaats en tijd anders zijn dan voorspelt. De plaats kan namelijk lastig bereikbaar zijn of midden in de nacht zijn. Daarnaast werkt de Data ook niet mee. Zo zijn er tijdbepalingen van het bereiken van aannemers, willekeurig ingevoerd, zodat je soms aannemer tijden na de storing tijden krijg waardoor de reparatietijd negatief wordt.

##### Succes ratio:

Dus de programma van eisen in een paar woorden:  
**S**pecifiek: Reparatietijd bepalen voor reizigers  
**M**eetbaar: Gebruik gemaakt van vorige meldingen Data  
**A**cceptabel: Het hebben van een voorspelling geeft de planners voldoening  
**R**ealistisch: Met de juiste Data en ingeschatte waardes is het haalbaar  
**T**ijdsgebonden: Zo snel mogelijk ingeschat worden (rond de 5 min)  

Deze voorspelling moet weergegeven worden in een applicatie (zie Userstory Application)

### Data understanding

We willen de hele process tijd weten totdat het mogelijk is om te rijden vanaf het moment dat het probleem in het systeem wordt vermeld, dus stm_aanntpl_tijd is het begin. Het einde moet zijn wanneer de treinen officieel kunnen rijden dus officiele eind tijd melding. 

Dus de columns: 
'stm_aanntpl_tijd' (begin tijd melding)
'stm_fh_ddt' (eind tijd melding)

We beginnen eerst met het uitlezen van de data en de duplicates weg te halen met behulp van meldingsnummer


In [ ]:
read_data = pd.read_csv('pro-rails_data.csv')
read_data = read_data.drop_duplicates(subset=['#stm_sap_meldnr'])
read_data = read_data.reset_index(drop=True)

Uit de dtypes kan je zien de tijden een object (string waarschijnlijk) zijn. Wat betekent dat deze kolommen niet gebruikt kan worden om te rekenen. We zetten ze om naar een datetime

In [ ]:
read_data ['stm_aanntpl_tijd']= pd.to_datetime(read_data ['stm_aanntpl_tijd'])
read_data ['stm_fh_ddt']= pd.to_datetime(read_data ['stm_fh_ddt'])

Daarna moet de reparatietijden uitgerekend worden dus eind - begin en als laatste de belangrijke niet wetende informatie rijen verwijderen:

In [ ]:
df = read_data.copy()
df['stm_hers_tijd'] = (df['stm_fh_ddt'].dt.hour * 60 + df['stm_fh_ddt'].dt.minute) - (df['stm_aanntpl_tijd'].dt.hour * 60 + df['stm_aanntpl_tijd'].dt.minute)

df = df.dropna(subset=['stm_hers_tijd','stm_prioriteit' ]) 
df = df.reset_index(drop=True)
df

,Unnamed: 0,#stm_sap_meldnr,stm_mon_nr,stm_vl_post,stm_sap_meld_ddt,stm_sap_mon_meld_ddt,stm_sap_meldtekst_lang,stm_mon_begin_ddt,stm_sap_meldtekst,stm_mon_toelichting_trdl,...,stm_rapportage_jaar,stm_x_bron_publ_dt,stm_x_bron_bestandsnaam,stm_x_bron_arch_dt,stm_x_actueel_ind,stm_x_run_id,stm_x_bk,stm_x_start_sessie_dt,stm_x_vervallen_ind,stm_hers_tijd
0,1,50053211,0.0,NaN,02/01/2006 09:00:00,02/01/2006 09:00:00,Logboeknr Geeltje : 49 Tijd: 0900 VL-Po...,NaN,Logboeknr Geeltje : 49 Tijd: 0900,NaN,...,2006,02/01/2006 09:00:00,\\PUHAPS0149\Informatica_Prod\Inbox\Informatic...,31/12/9999 00:00:00,1,1518,12704590,13/06/2013 13:37:52,0,540.0
1,2,50053213,48.0,GN,02/01/2006 12:35:00,02/01/2006 12:35:00,Logboeknr RBV : 48 Tijd: 1235 VL-Post: ...,02/01/2006 12:35:00,Logboeknr RBV : 48 Tijd: 1235 VL-P,NaN,...,2006,02/01/2006 12:35:00,\\PUHAPS0149\Informatica_Prod\Inbox\Informatic...,31/12/9999 00:00:00,1,1518,12704591,13/06/2013 13:37:52,0,806.0
2,3,50053214,72.0,ZL,02/01/2006 16:40:00,02/01/2006 16:40:00,Logboeknr RBV : 72 Tijd: 1640 VL-Post: ...,02/01/2006 16:40:00,Logboeknr RBV : 72 Tijd: 1640 VL-P,NaN,...,2006,02/01/2006 16:40:00,\\PUHAPS0149\Informatica_Prod\Inbox\Informatic...,31/12/9999 00:00:00,1,1518,12704592,13/06/2013 13:37:52,0,1040.0
3,4,50053215,96.0,ZL,02/01/2006 22:30:00,02/01/2006 22:30:00,Logboeknr RBV : 96 Tijd: 2230 VL-Post: ...,02/01/2006 22:30:00,Logboeknr RBV : 96 Tijd: 2230 VL-P,NaN,...,2006,02/01/2006 22:30:00,\\PUHAPS0149\Informatica_Prod\Inbox\Informatic...,31/12/9999 00:00:00,1,1518,12704593,13/06/2013 13:37:52,0,1356.0
4,5,50053218,38.0,EHV,02/01/2006 11:23:00,02/01/2006 11:23:00,Logboeknr RBV : 38 Tijd: 1123 VL-Post: ...,02/01/2006 11:23:00,Logboeknr RBV : 38 Tijd: 1123 VL-P,NaN,...,2006,02/01/2006 11:23:00,\\PUHAPS0149\Informatica_Prod\Inbox\Informatic...,31/12/9999 00:00:00,1,1518,12704594,13/06/2013 13:37:52,0,690.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461153,908620,99003503,0.0,NaN,11/05/2013 07:55:00,11/05/2013 07:55:00,Ldm-Akl : km-60.0 ATB code 4,NaN,Ldm-Akl : km-60.0 ATB code 4,NaN,...,2013,11/05/2013 07:55:00,\\PUHAPS0149\Informatica_Prod\Inbox\Informatic...,05/08/2013 01:00:16,0,1518,12759541,13/06/2013 13:37:52,0,179.0
461154,908622,99003504,0.0,NaN,11/05/2013 07:59:00,11/05/2013 07:59:00,A15 : km 102.6 deur geluidsscherm open,NaN,A15 : km 102.6 deur geluidsscherm open,NaN,...,2013,11/05/2013 07:59:00,\\PUHAPS0149\Informatica_Prod\Inbox\Informatic...,05/08/2013 01:00:16,0,1518,12759542,13/06/2013 13:37:52,0,3.0
461155,908624,99003505,0.0,NaN,11/05/2013 08:06:00,11/05/2013 08:06:00,GVC : sp-2 en 3 stop lampen op de stootjukke...,NaN,GVC : sp-2 en 3 stop lampen op de stoot,NaN,...,2013,11/05/2013 08:06:00,\\PUHAPS0149\Informatica_Prod\Inbox\Informatic...,05/08/2013 01:00:16,0,1518,12759543,13/06/2013 13:37:52,0,24.0
461156,908626,99003506,0.0,NaN,11/05/2013 09:21:00,11/05/2013 09:21:00,Whz : Ovw 100.1 albert Plesmanweg blijft dic...,NaN,Whz : Ovw 100.1 albert Plesmanweg blijft,NaN,...,2013,11/05/2013 09:21:00,\\PUHAPS0149\Informatica_Prod\Inbox\Informatic...,05/08/2013 01:00:16,0,1518,12759544,13/06/2013 13:37:52,0,36.0


Nu gaan we het gemiddelde berekenen van de hersteltijd

In [ ]:
oplostijd_data = pd.DataFrame(df['stm_hers_tijd'])
oplostijd_data

,stm_hers_tijd
0,540.0
1,806.0
2,1040.0
3,1356.0
4,690.0
...,...
461153,179.0
461154,3.0
461155,24.0
461156,36.0


In [ ]:
gemiddelde_oplostijd = oplostijd_data.mean()
gemiddelde_oplostijd = round(float(str(gemiddelde_oplostijd)[15:25]))
print("Gemiddelde: " + str(gemiddelde_oplostijd) + " minuten")
oplostijd_data['gemiddelde'] = gemiddelde_oplostijd
oplostijd_data

Gemiddelde: 79 minuten


,stm_hers_tijd,gemiddelde
0,540.0,79
1,806.0,79
2,1040.0,79
3,1356.0,79
4,690.0,79
...,...,...
461153,179.0,79
461154,3.0,79
461155,24.0,79
461156,36.0,79


In [ ]:
r2_score(oplostijd_data['gemiddelde'], oplostijd_data['stm_hers_tijd'])

0.0

In [ ]:
mean_squared_error(oplostijd_data['gemiddelde'], oplostijd_data['stm_hers_tijd'], squared=False)

294.1136599120333

Een RMSE van bijna 300 minuten is erg veel. Er moet nog veel gedaan worden met de data om het beter in te schatten